# Exploration des tests en fatigue des matériaux
## Prédiction de la courbe S-N avec un réseau de neurone informé

In [ ]:
import os
import random

import pandas as pd       
import numpy as np  

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score, mean_squared_error, median_absolute_error, mean_absolute_percentage_error

from scipy.stats import spearmanr

import tensorflow as tf
import tensorflow_probability as tfp

from tensorflow import keras
from tensorflow.keras.losses import Loss

In [ ]:
SEED = 42 

def set_seeds(seed: int = SEED) -> None:
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    keras.utils.set_random_seed(seed)
    tf.experimental.numpy.random.seed(seed)
    # When using the CuDNN backend (CUDA), two additional options must be configured.
    #os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
    #os.environ['TF_DETERMINISTIC_OPS'] = '1'
    print(f"Random seed set as {seed}")

def set_global_determinism(seed=SEED):
    set_seeds(seed=seed)

    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
    
    tf.config.threading.set_inter_op_parallelism_threads(1)
    tf.config.threading.set_intra_op_parallelism_threads(1)

set_global_determinism(seed=SEED)

# A propos des données de fatigue

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/Mileem/PFIA_2024/main/data/raw/AAW.csv', delimiter=';', decimal=',', index_col=0, skiprows=[0]) 

df.head()

In [ ]:
df.info()

##  Conversion au logarithme

In [ ]:
# TODO: Convert Stress and life cycle into logarithms


In [ ]:
df.head()

## Visualisation des données

In [ ]:
g = sns.relplot(data=df, x="life_log", y="stress_log_mpa")
g.set_axis_labels("Fatigue life (cycles) log-scale", "Stress Amplitude (MPa) log-scale", labelpad=10)

# Preprocessing

## Préparation des données avant utilisation

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(df[['stress_log_mpa']].values, 
                                                  df['life_log'], test_size=0.2)

(x_train, x_val, y_train, y_val) = (tf.convert_to_tensor(x_train, dtype=tf.float32),tf.convert_to_tensor(x_val, dtype=tf.float32), 
 tf.convert_to_tensor(y_train, dtype=tf.float32), tf.convert_to_tensor(y_val, dtype=tf.float32))

### Visualisation des données d'entrainement et de validation

In [ ]:
def plot_dataset(x_train, y_train, x_val, y_val, title):
    fig = plt.figure(figsize = (10, 8))
    plt.scatter(y_train, x_train, marker='+', label='Train')
    plt.scatter(y_val, x_val, marker='+', color='r', label='Val')

    plt.title(title)
    plt.xlabel('Fatigue life (cycles) log-scale')
    plt.ylabel('Stress Amplitude (MPa) log-scale')
    plt.legend()
    plt.show()
    
plot_dataset(x_train, y_train, x_val, y_val, 'Dataset')

## Création d'un nouveau jeu de donnée de test

In [ ]:
x_test = np.linspace(df['stress_log_mpa'].min(), 
                                      df['stress_log_mpa'].max()+0.01, 100).reshape(-1, 1)

# Physics-Informed Neural Networks
## Création de l'architecture du réseau de neurone

In [ ]:
tfkl = tf.keras.layers
tfkm = tf.keras.models
tfpl = tfp.layers
tfd = tfp.distributions


def nn_architecture(lr,loss_function): 
    # TODO: Building the neural network architecture

    return model

def model_fit(model, x_train, y_train, x_val, y_val, epochs):
    model.fit(x_train, y_train, validation_data=(x_val, y_val), 
                        epochs=epochs, verbose=False, shuffle=True, batch_size = 30)
    
    return model

## Création de la Loss fonction

In [ ]:
def loss_knowledge():
    #TODO: Create knowledge loss function
    
    return loss_knowledge_

def custom_loss(y_true, y_pred, lambda_penalty):
    negloglik = #TODO: Add negative log likelihood loss
    return negloglik #TODO

In [ ]:
class CustomLoss(Loss):
    def __init__(self):
        super().__init__()
        #TODO Create custom loss

    def call(self, y_true, y_pred):
        return #TODO return Custom loss

## Entrainement du réseau de neurones

In [ ]:
def create_model(epochs, lr, lambda_penalty, x_collocation) :
    loss_fn = #TODO
    model = #TODO
    return model

In [ ]:
# Model Architecture
epochs = #TODO
lr = #TODO
lambda_penalty = #TODO

model = create_model(#TODO)
model = model_fit(#TODO)

## Résultats 

In [ ]:
y_pred = model(x_test)
y_pred_mean = y_pred.mean()
y_pred_std = y_pred.stddev()

## Visualisation de la Courbe S-N

In [ ]:
fig = plt.figure(figsize = (10, 8))
plt.scatter(y_train, x_train, marker='+', label='Train')
plt.plot(y_pred_mean, x_test, color='r', label='Predicted Mean')
plt.fill_betweenx(x_test.ravel(), np.array(y_pred_mean+1.96*y_pred_std).ravel(), np.array(y_pred_mean-1.96*y_pred_std).ravel(), x_test.ravel(), color='C1', alpha=0.5, label='95% Confidence Level')
plt.title(f'PINN on fatigue life cycles, epochs={epochs}, learning rate={lr}, 95% Confidence Level')
plt.xlabel('Fatigue life (cycles) log-scale')
plt.ylabel('Stress Amplitude (MPa) log-scale')
plt.legend()
plt.show()

## Evaluation des résultats obtenus
### Métriques habituelles

In [ ]:
predict_val = model.predict(x_val)
r2 = explained_variance_score(y_val, predict_val)
rmse = mean_squared_error(y_val, predict_val, squared=False)
mape = mean_absolute_percentage_error(y_val, predict_val)
print('R2: ', r2)
print('RMSE: ', rmse)
print('MAPE: ', mape)

### Métriques liées aux connaissances (contexte) de la fatigue des matériaux

In [ ]:
def evaluate_spearman_std(model, x_test):
    #TODO
    return spearman_std, p_value_std

def evaluate_spearman_mean(model, x_test):
    #TODO
    return spearman_mean, p_value_mean

def evaluate_spearman_curve(model, x_test):
    #TODO
    return spearman_curve, p_value_curve 

### Visualisation des coefficients de corrélations de Spearman

In [ ]:
dx_dt = np.gradient(x_test[:, 0])
dy_dt = np.gradient(y_pred_mean[:, 0])

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(13, 5))
fig.suptitle('Summarize of model')
ax1.plot(x_test, y_pred_std, color='r', label='Predicted Standard Deviation')
ax1.title.set_text('Standard deviation')
ax1.set_xlabel('Stress Amplitude (MPa) log-scale')
ax1.set_ylabel('Standard Deviation')

ax2.plot(x_test, y_pred_mean, color='r', label='Predicted Mean')
ax2.scatter(x_train, y_train, marker='+', label='Train')
ax2.title.set_text('Mean')
ax2.set_xlabel('Stress Amplitude (MPa) log-scale')
ax2.set_ylabel('Mean')

ax3.plot(x_test, (dy_dt/dx_dt), color='r', label='Predicted Curvature')
ax3.title.set_text('Curvature')
ax3.set_xlabel('Stress Amplitude (MPa) log-scale')
ax3.set_ylabel('Curvature')

# Sauvegarder la figure dans un fichier PNG
#plt.savefig('img/results.png', format='png')

In [ ]:
spearman_mean, p_value_mean = evaluate_spearman_mean(model, x_test)
spearman_std, p_value_std = evaluate_spearman_std(model, x_test)
spearman_curve, p_value_curve = evaluate_spearman_curve(model, x_test)
print({'Spearman Std': spearman_std, 'p_value std': p_value_std, 'Spearman Mean': spearman_mean, 'p_value mean': p_value_mean, 'Spearman Curve': spearman_curve, 'p_value curve': p_value_curve})

# Evaluation de plusieurs modèles

In [ ]:
model_configs = [
    {'epochs': 800, 'lr': 5e-3, 'lambda': 0, 'x_collocation': x_collocation},
    {'epochs': 800, 'lr': 5e-3, 'lambda': 100, 'x_collocation': x_collocation},
    {'epochs': 800, 'lr': 5e-3, 'lambda': 1000, 'x_collocation': x_collocation},
    {'epochs': 1000, 'lr': 5e-3, 'lambda': 0, 'x_collocation': x_collocation},
    {'epochs': 1000, 'lr': 5e-3, 'lambda': 1000, 'x_collocation': x_collocation}
]

results = []

for i, config in enumerate(model_configs):
    model = create_model(#TODO)
    model = model_fit(#TODO)

    spearman_mean, p_value_mean = evaluate_spearman_mean(model, x_test)
    spearman_std, p_value_std = evaluate_spearman_std(model, x_test)
    spearman_curve, p_value_curve = evaluate_spearman_curve(model, x_test)
    results.append({'Model': f'Model {i+1}', 'Spearman Std': spearman_std, 'p_value std': p_value_std, 'Spearman Mean': spearman_mean, 'p_value mean': p_value_mean, 'Spearman Curve': spearman_curve, 'p_value curve': p_value_curve})

results_df = pd.DataFrame(results)

results_df